In [1]:
import math
import numpy as np
from Bio.PDB import PDBParser

1. Lire la structure PDB et extraire les atomes de la chaîne principale (N, CA, C, O)

In [1]:
class ReadPdbStructure:
    """RReprésenter et extraire les atomes de la chaîne principale (N, CA, C, O) dans la structure PDB."""

    def __init__(self, pdb_filename):
        self.pdb_filename = pdb_filename
        self.structure = self._read_pdb_structure()
        self.get_main_chain_atomes = self._get_main_chain_atomes()

    def _read_pdb_structure(self):
        """Lire le fichier PDB et renvoyer la structure"""
        parser = PDBParser()
        structure = parser.get_structure("protein",self.pdb_filename)
        return structure
    
    def _get_main_chain_atomes(self):
        """Extaire les coordonnées"""
        main_chain_atoms = []
        for model in self.structure:
            for chain in model:
                for residue in chain:
                    atom_dict = {}
                    """Stocker le résidu actuel dans le dictionnaire"""
                    atom_dict['residue'] = residue  
                    for atom in residue.get_atoms():
                        if atom.get_name() in ["N","CA", "C", "O"]:
                            atom_dict[atom.get_name()] = atom.get_vector()
                    main_chain_atoms.append(atom_dict)
        return main_chain_atoms
    
    def get_backbone_atoms(self):
        """Obtenir des informations atomiques sur la chaîne principale"""
        return self.backbone_atoms
        

2. Calculer des hydrogènes 

In [1]:
class HydrogenBondDetector:
    """Identifier les liaisons hydrogène en fonction de l'énergie"""

    def __init__(self, main_chain_atoms):
        self.main_chain_atoms = main_chain_atoms

    def _calculate_hydrogen_bond_energy(self, donor_residue, acceptor_residue):
        """Calculer l'énergie d'une liaison hydrogène à partir de l'équation énergétique"""
        #Obtenir les coordonnées atomiques souhaitées
        O = acceptor_residue['O']
        N = donor_residue["N"]
        C = donor_residue["C"]
        #Calculer des distances entre les atoms
        r_ON = (O - N).norm()
        r_CN = (C -N).norm()
        # Simplified energy calculation (hydrogen atoms are not used)
        energy = 0.084 * (1/r_ON - 1/r_CN) * 332
        
        return energy


In [ ]:
class SecondaryStructureAssigner:
    """Secondary Structure Dispenser, Détection de α-Helix et β-Folding basée sur la liaison hydrogène"""
